# 6.10.1 Ecuaciones diferenciales de compatibilidad tridimensionales

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Diego Andrés Alvarez, <daalvarez@unal.edu.co>  | Marzo 26, 2023 | Código inicial |

## Ecuaciones de compatibilidad de Saint-Venant

In [1]:
from sympy import sin, cos, diff, Function, simplify, symbols, init_printing, latex, Matrix, expand

# Para imprimir bonito
init_printing()
from IPython.display import Math                 
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

Se definen las variables y funciones simbólicas necesarias:

In [2]:
r, t, z, E, G, nu = symbols("r, theta, z, E, G, nu")

sr  = Function('sigma_r')(r, t, z)
st  = Function('sigma_θ')(r, t, z)
sz  = Function('sigma_z')(r, t, z)
trt = Function('tau_rθ')(r, t, z)
trz = Function('tau_rz')(r, t, z)
ttz = Function('tau_θz')(r, t, z)

er  = Function('varepsilon_r')(r, t, z)
et  = Function('varepsilon_θ')(r, t, z)
ez  = Function('varepsilon_z')(r, t, z)
grt = Function('gamma_rθ')(r, t, z);        ert = grt/2
grz = Function('gamma_rz')(r, t, z);        erz = grz/2
gtz = Function('gamma_θz')(r, t, z);        etz = gtz/2

Se define la matriz de tensiones en coordenadas cilíndricas $\underline{\underline{\boldsymbol{\sigma}}}_{\text{cil}}$:

In [3]:
eps_cil = Matrix([[  er, ert, erz ],
                  [ ert,  et, etz ],                       
                  [ erz, etz,  ez ]])

Se define la matriz de transformación $\boldsymbol{T}$:

In [4]:
T = Matrix([[ cos(t), -sin(t),       0 ],     
            [ sin(t),  cos(t),       0 ],      
            [      0,       0,       1 ]])

Se calcula la matriz de deformaciones $\underline{\underline{\boldsymbol{\varepsilon}}}_{\text{cil}}$ en el sistema de coordenadas especificadas por los vectores definidos en la matriz $\boldsymbol{T}$. Recuerde que $$\underline{\underline{\boldsymbol{\varepsilon}}} = \boldsymbol{T} \underline{\underline{\boldsymbol{\varepsilon}}}_{\text{cil}} \boldsymbol{T}^T.$$

In [5]:
eps = T*eps_cil*T.T            # ecuación (6.39)

Se extraen los términos de la matriz de deformaciones $\underline{\underline{\boldsymbol{\varepsilon}}}$:

In [6]:
ex  = eps[0,0]
ey  = eps[1,1]
ez  = eps[2,2]
gxy = 2*eps[0,1]
gxz = 2*eps[0,2]
gyz = 2*eps[1,2]

In [7]:
# se definen las primeras derivadas utilizando la regla de la cadena
d_dx   = lambda f: diff(f,r)*cos(t) - diff(f,t)*sin(t)/r
d_dy   = lambda f: diff(f,r)*sin(t) + diff(f,t)*cos(t)/r
d_dz   = lambda f: diff(f,z)

# se definen las segundas derivadas
d2_dx2 = lambda f: d_dx(d_dx(f))
d2_dy2 = lambda f: d_dy(d_dy(f))
d2_dz2 = lambda f: d_dz(d_dz(f))

d2_dxdy = lambda f: d_dx(d_dy(f))
d2_dxdz = lambda f: d_dx(d_dz(f))
d2_dydz = lambda f: d_dy(d_dz(f))

Ecuaciones de compatibilidad de Saint-Venant en coordenadas rectangulares son:
\begin{align}
\frac{\partial^2 \varepsilon_x}{\partial y^2}+\frac{\partial^2 \varepsilon_y}{\partial x^2} &= \frac{\partial^2 \gamma_{xy}}{\partial x \partial y} &  2\frac{\partial^2 \varepsilon_x}{\partial y \partial z} &= \frac{\partial}{\partial x}\left(-\frac{\partial \gamma_{yz}}{\partial x}+\frac{\partial \gamma_{xz}}{\partial y}+\frac{\partial \gamma_{xy}}{\partial z}\right) \\
\frac{\partial^2 \varepsilon_y}{\partial z^2}+\frac{\partial^2 \varepsilon_z}{\partial y^2} &= \frac{\partial^2 \gamma_{yz}}{\partial y \partial z} &  2\frac{\partial^2 \varepsilon_y}{\partial x \partial z} &= \frac{\partial}{\partial y}\left(\frac{\partial \gamma_{yz}}{\partial x}-\frac{\partial \gamma_{xz}}{\partial y}+\frac{\partial \gamma_{xy}}{\partial z}\right) \\
\frac{\partial^2 \varepsilon_z}{\partial x^2}+\frac{\partial^2 \varepsilon_x}{\partial z^2} &= \frac{\partial^2 \gamma_{xz}}{\partial x \partial z} &  2\frac{\partial^2 \varepsilon_z}{\partial x \partial y} &= \frac{\partial}{\partial z}\left(\frac{\partial \gamma_{yz}}{\partial x}+\frac{\partial \gamma_{xz}}{\partial y}-\frac{\partial \gamma_{xy}}{\partial z}\right);
\end{align}
(tenga en cuenta que estas ecuaciones se escribieron de modo tal que sus lados derechos son iguales a cero)

In [8]:
e1 = d2_dy2(ex) + d2_dx2(ey) - d2_dxdy(gxy)
e2 = d2_dz2(ey) + d2_dy2(ez) - d2_dydz(gyz)
e3 = d2_dx2(ez) + d2_dz2(ex) - d2_dxdz(gxz)
e4 = 2*d2_dydz(ex) - d_dx(- d_dx(gyz) + d_dy(gxz) + d_dz(gxy))
e5 = 2*d2_dxdz(ey) - d_dy(+ d_dx(gyz) - d_dy(gxz) + d_dz(gxy))
e6 = 2*d2_dxdy(ez) - d_dz(+ d_dx(gyz) + d_dy(gxz) - d_dz(gxy))

In [9]:
# Se hace sin(t)=0 y cos(t)=1 en las anteriores ecuaciones
e1 = simplify(e1.subs({sin(t): 0, cos(t): 1})).expand()
e2 = simplify(e2.subs({sin(t): 0, cos(t): 1})).expand()
e3 = simplify(e3.subs({sin(t): 0, cos(t): 1})).expand()
e4 = simplify(e4.subs({sin(t): 0, cos(t): 1})).expand()
e5 = simplify(e5.subs({sin(t): 0, cos(t): 1})).expand()
e6 = simplify(e6.subs({sin(t): 0, cos(t): 1})).expand()

Las ecuaciones de Saint-Venant que aparecen en la literatura son:
\begin{gather}
        \frac{1}{r^2}\frac{\partial^2 \varepsilon_r}{\partial \theta^2} + \frac{1}{r}\frac{\partial}{\partial r} \left( r \frac{\partial \varepsilon_\theta}{\partial r} - \left( \varepsilon_r - \varepsilon_\theta \right) \right) = \frac{1}{r}\frac{\partial}{\partial \theta}\left(\frac{\partial \gamma_{r\theta}}{\partial r} + \frac{\gamma_{r \theta}}{r} \right) \\
        %
        %\allowdisplaybreaks
        \frac{1}{r^2}\frac{\partial^2 \varepsilon_z}{\partial \theta^2} + \frac{\partial^2 \varepsilon_\theta}{\partial z^2} + \frac{1}{r}\frac{\partial \varepsilon_z}{\partial r} = \frac{1}{r}\frac{\partial}{\partial z} \left( \frac{\partial \gamma_{\theta z}}{\partial \theta} + \gamma_{rz} \right) \\
        %
        %\allowdisplaybreaks
        \frac{\partial^2 \varepsilon_z}{\partial r^2} + \frac{\partial^2 \varepsilon_r}{\partial z^2} = \frac{\partial^2 \gamma_{rz}}{\partial r \partial z} \\
        %
        %\allowdisplaybreaks
        \frac{2}{r}\frac{\partial^2 \varepsilon_r}{\partial \theta \partial z} = 
        \frac{1}{r} \frac{\partial}{\partial r} \left( r\frac{\partial \gamma_{r\theta}}{\partial z} - \gamma_{\theta z} \right) + \frac{\partial}{\partial r} \left( \frac{1}{r}\frac{\partial \gamma_{rz}}{\partial \theta} - \frac{\partial\gamma_{\theta z}}{\partial r} \right) + \frac{
            1
        }{r}\frac{\partial \gamma_{r\theta}}{\partial z} + \frac{\gamma_{\theta z}}{r^2} \\
        %
        2\frac{\partial}{\partial z} \left( \frac{\partial \varepsilon_\theta}{\partial r} - \left( \frac{\varepsilon_r - \varepsilon_\theta}{r} \right) \right) = \frac{1}{r}\frac{\partial}{\partial \theta} \left( \frac{\partial \gamma_{\theta z}}{\partial r} - \frac{1}{r}\frac{\partial \gamma_{rz}}{\partial \theta} + \frac{\partial \gamma_{r \theta}}{\partial z} \right) + \frac{1}{r^2}\frac{\partial \gamma_{\theta z}}{\partial \theta} \\
        %
        \frac{2}{r}\frac{\partial}{\partial \theta} \left( \frac{\partial \varepsilon_z}{\partial r} - \frac{\varepsilon_z}{r} \right) = \frac{\partial}{\partial z} \left( \frac{\partial \gamma_{\theta z}}{\partial r} + \frac{1}{r}\frac{\partial \gamma_{rz}}{\partial \theta} - \frac{\partial \gamma_{r \theta}}{\partial z} - \frac{\gamma_{\theta z}}{r} \right).
\end{gather}

In [10]:
# Se verifica que las ecuaciones calculadas son iguales a las ecuaciones Saint-Venant que aparecen en la literatura:
ee1 = (1/r**2)*diff(er,t,2) + (1/r)*diff(r*diff(et,r) - (er-et),r) - (1/r)*diff(diff(grt,r)  + grt/r,t)
ee2 = (1/r**2)*diff(ez,t,2) + diff(et,z,2) + (1/r)*diff(ez,r) - (1/r)*diff(diff(gtz,t) + grz,z)
ee3 = diff(ez,r,2) + diff(er,z,2) - diff(grz,r,1,z,1)
ee4 = (2/r)*diff(er,t,1,z,1) - (1/r)*diff(r*diff(grt,z) - gtz,r) - diff((1/r)*diff(grz,t) - diff(gtz,r),r) - (1/r)*diff(grt,z) - gtz/r**2
ee5 = 2*diff(diff(et,r) - (er-et)/r,z) - (1/r)*diff(diff(gtz,r) - (1/r)*diff(grz,t) + diff(grt,z),t) - (1/r**2)*diff(gtz,t)
ee6 = (2/r)*diff(diff(ez,r) - ez/r,t) - diff(diff(gtz,r) + (1/r)*diff(grz,t) - diff(grt,z) - gtz/r,z)

Observe que las siguientes restan dan cero, lo que confirma que calculamos las ecuaciones de Saint-Venant:

In [11]:
expand(e1 - ee1)

In [12]:
expand(e2 - ee2)

In [13]:
expand(e3 - ee3)

In [14]:
expand(e4 - ee4)

In [15]:
expand(e5 - ee5)

In [16]:
expand(e6 - ee6)

In [17]:
imprimir("", e1, " = 0")

<IPython.core.display.Math object>

In [18]:
imprimir("", e2, " = 0")

<IPython.core.display.Math object>

In [19]:
imprimir("", e3, " = 0")

<IPython.core.display.Math object>

In [20]:
imprimir("", e4, " = 0")

<IPython.core.display.Math object>

In [21]:
imprimir("", e5, " = 0")

<IPython.core.display.Math object>

In [22]:
imprimir("", e6, " = 0")

<IPython.core.display.Math object>